In [33]:
import os
import duckdb
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(usecwd=False))
BASE_DIR = Path(os.environ["BASE_DIR"])
print(f'Base dir: {BASE_DIR}')

DB_PATH = f"{BASE_DIR}/data/reddit.duckdb"

Base dir: /home/martina/Desktop/Git/reddit-llm


# Table of Contents
* [Import tickers](#import_tickers)
    * [Section 1.1](#section_1_1)
* [DB Queries](#db_queries)
    * [Section 2.1](#section_2_1)
        * [Sub Section 2.1.1](#sub_section_2_1_1)
        * [Sub Section 2.1.2](#sub_section_2_1_2)

## Import tickers (one-off) <a class="anchor" id="import_tickers"></a>

In [34]:
# Write connection
con = duckdb.connect(str(DB_PATH))

In [27]:
import pandas as pd

df_nasdaq = pd.read_csv('https://datahub.io/core/nasdaq-listings/r/nasdaq-listed-symbols.csv')
df_nyse = pd.read_csv('https://datahub.io/core/nyse-other-listings/r/nyse-listed.csv')

display(df_nasdaq)

,Symbol,Company Name,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
0,AACB,Artius II Acquisition Inc.,Artius II Acquisition Inc. - Class A Ordinary ...,G,N,N,100.0,N,N
1,AACBR,Artius II Acquisition Inc.,Artius II Acquisition Inc. - Rights,G,N,N,100.0,N,N
2,AACBU,Artius II Acquisition Inc.,Artius II Acquisition Inc. - Units,G,N,N,100.0,N,N
3,AACG,ATA Creativity Global,ATA Creativity Global - American Depositary Sh...,S,N,N,100.0,N,N
4,AADR,AdvisorShares Dorsey Wright ADR ETF,AdvisorShares Dorsey Wright ADR ETF,G,N,N,100.0,Y,N
...,...,...,...,...,...,...,...,...,...
5247,ZXZZT,NASDAQ TEST STOCK,NASDAQ TEST STOCK,G,Y,N,100.0,N,N
5248,ZYBT,Zhengye Biotechnology Holding Limited,Zhengye Biotechnology Holding Limited - Ordina...,S,N,N,100.0,N,N
5249,ZYME,Zymeworks Inc.,Zymeworks Inc. - Common Stock,Q,N,N,100.0,N,N
5250,File Creation Time: 1231202518:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Remove the last 2 weird rows
df_nasdaq = df_nasdaq[:-2]

In [5]:
display(df_nyse)

,ACT Symbol,Company Name
0,A,"Agilent Technologies, Inc. Common Stock"
1,AA,Alcoa Corporation Common Stock
2,AAM,AA Mission Acquisition Corp. Class A Ordinary ...
3,AAM.U,"AA Mission Acquisition Corp. Units, each consi..."
4,AAM.W,"AA Mission Acquisition Corp. Warrants, each wh..."
...,...,...
2879,ZTO,ZTO Express (Cayman) Inc. American Depositary ...
2880,ZTR,Virtus Total Return Fund Inc.
2881,ZTS,Zoetis Inc. Class A Common Stock
2882,ZVIA,Zevia PBC Class A Common Stock


In [9]:
# Rename ticker to match the other table
df_nyse.rename(columns={'ACT Symbol': 'Symbol'}, inplace=True)
df_nyse

,Symbol,Company Name
0,A,"Agilent Technologies, Inc. Common Stock"
1,AA,Alcoa Corporation Common Stock
2,AAM,AA Mission Acquisition Corp. Class A Ordinary ...
3,AAM.U,"AA Mission Acquisition Corp. Units, each consi..."
4,AAM.W,"AA Mission Acquisition Corp. Warrants, each wh..."
...,...,...
2879,ZTO,ZTO Express (Cayman) Inc. American Depositary ...
2880,ZTR,Virtus Total Return Fund Inc.
2881,ZTS,Zoetis Inc. Class A Common Stock
2882,ZVIA,Zevia PBC Class A Common Stock


In [10]:
df_all_tickers = pd.concat([df_nasdaq, df_nyse], ignore_index=True)
df_all_tickers

,Symbol,Company Name,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
0,AACB,Artius II Acquisition Inc.,Artius II Acquisition Inc. - Class A Ordinary ...,G,N,N,100.0,N,N
1,AACBR,Artius II Acquisition Inc.,Artius II Acquisition Inc. - Rights,G,N,N,100.0,N,N
2,AACBU,Artius II Acquisition Inc.,Artius II Acquisition Inc. - Units,G,N,N,100.0,N,N
3,AACG,ATA Creativity Global,ATA Creativity Global - American Depositary Sh...,S,N,N,100.0,N,N
4,AADR,AdvisorShares Dorsey Wright ADR ETF,AdvisorShares Dorsey Wright ADR ETF,G,N,N,100.0,Y,N
...,...,...,...,...,...,...,...,...,...
8129,ZTO,ZTO Express (Cayman) Inc. American Depositary ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8130,ZTR,Virtus Total Return Fund Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8131,ZTS,Zoetis Inc. Class A Common Stock,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8132,ZVIA,Zevia PBC Class A Common Stock,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
con.sql('''CREATE TABLE tickers AS 
              SELECT DISTINCT Symbol as ticker
              FROM df_all_tickers''')

In [14]:
con.table("tickers").show()

┌────────────┐
│   ticker   │
│  varchar   │
├────────────┤
│ AALG       │
│ AAOI       │
│ AAON       │
│ AAPU       │
│ ABNG       │
│ ABTS       │
│ ABUS       │
│ ACWX       │
│ ADAMN      │
│ ADBE       │
│  ·         │
│  ·         │
│  ·         │
│ UTF        │
│ VIPS       │
│ VLTO       │
│ VNO$M      │
│ WD         │
│ WFC        │
│ WH         │
│ WLKP       │
│ XYF        │
│ ZETA       │
├────────────┤
│ 8133 rows  │
│ (20 shown) │
└────────────┘



## DB Queries <a class="anchor" id="db_queries"></a>

In [81]:
# Read connection
con = duckdb.connect(str(DB_PATH), read_only = True)

In [82]:
con.sql('SHOW ALL TABLES;')

┌──────────┬─────────┬─────────────────┬─────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────┬───────────┐
│ database │ schema  │      name       │                            column_names                             │                           column_types                            │ temporary │
│ varchar  │ varchar │     varchar     │                              varchar[]                              │                             varchar[]                             │  boolean  │
├──────────┼─────────┼─────────────────┼─────────────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────┼───────────┤
│ reddit   │ main    │ comment_tickers │ [comment_id, ticker, direction, confidence, method]                 │ [VARCHAR, VARCHAR, VARCHAR, DOUBLE, VARCHAR]                      │ false     │
│ reddit   │ main    │ comments        │ [com

In [83]:
con.sql('''SELECT * FROM comment_tickers LIMIT 100;''').show()

┌────────────┬─────────┬───────────┬────────────┬──────────────┐
│ comment_id │ ticker  │ direction │ confidence │    method    │
│  varchar   │ varchar │  varchar  │   double   │   varchar    │
├────────────┼─────────┼───────────┼────────────┼──────────────┤
│ nyl0vvt    │ DD      │ neutral   │        0.0 │ caps_context │
│ nylf1cy    │ GOOG    │ neutral   │        0.0 │ caps_plain   │
│ nylf1cy    │ AAPL    │ neutral   │        0.0 │ caps_plain   │
│ nyl98cq    │ AAPL    │ neutral   │        0.5 │ caps_plain   │
│ nylbxlb    │ AAPL    │ bearish   │        1.0 │ caps_context │
│ nyma4la    │ AAPL    │ neutral   │        0.0 │ caps_plain   │
│ nymdxho    │ GOOG    │ bullish   │        1.0 │ caps_context │
│ nymdxho    │ AAPL    │ bullish   │        1.0 │ caps_context │
│ nymfv5y    │ AAPL    │ neutral   │        0.0 │ caps_plain   │
│ nymlqx8    │ INTC    │ neutral   │        0.0 │ dollar       │
│    ·       │  ·      │    ·      │         ·  │   ·          │
│    ·       │  ·      │ 

In [84]:
posts = con.sql("SELECT * FROM posts").df()
display(posts)
# posts.to_clipboard()

,post_id,subreddit,title,body,score,num_comments,created_utc
0,1q61elv,wallstreetbets,Going balls deep on GOOG thanks to insiders on...,There was a post here a few weeks ago of the G...,2621,415,2026-01-07 01:15:39
1,1q6fi6o,wallstreetbets,"Warner Bros rejects $77.9B Paramount bid, back...",Source: [https://finance.yahoo.com/news/warner...,1848,138,2026-01-07 13:35:08
2,1q5wqba,wallstreetbets,"Sandisk +27%, Western Digital +16%, Seagate +1...",Source: [https://finance.yahoo.com/news/sandis...,1503,281,2026-01-06 22:07:36
3,1q60s7e,wallstreetbets,Chat platform Discord files confidentially for...,,1325,272,2026-01-07 00:48:53
4,1q62lml,wallstreetbets,"PLTR is the Only Stock Worth >$400B, without A...",Position: 12/18/2026 200P @48.5,980,137,2026-01-07 02:07:17
...,...,...,...,...,...,...,...
204,1q94iur,ValueInvesting,Iran & Venezuela and the opportunities they cr...,Iran & Venezuela are both oil titans…in the ne...,2,3,2026-01-10 15:49:25
205,1q98pe7,ValueInvesting,HRB value trap?,Looking at H&R Block. They have earrings upcom...,1,3,2026-01-10 18:39:00
206,1q98fl5,ValueInvesting,PayPal in 2026: Value Trap or a Great Opportun...,"For a stock that has been struggling, what ins...",0,0,2026-01-10 18:28:35
207,1q8ezhl,ValueInvesting,How do you guys use AI to help investing?,I just started investing individual stocks las...,61,71,2026-01-09 19:45:55


In [86]:
comments = con.sql("SELECT * FROM comments").df()
display(comments)

,comment_id,post_id,parent_id,body,score,created_utc
0,ny4pzbq,1q61elv,t3_1q61elv,"Gambling on gambling, nice. How do I gamble on...",2102,2026-01-07 02:52:21
1,ny4b09v,1q61elv,t3_1q61elv,Why not just load up on those polymarket contr...,1952,2026-01-07 01:31:11
2,ny4jcy6,1q61elv,t3_1q61elv,Listen buddy. Google is a train. It took a lot...,106,2026-01-07 02:16:25
3,ny4dio8,1q61elv,t3_1q61elv,Didn’t this shit already happen?,101,2026-01-07 01:44:56
4,ny4ajee,1q61elv,t3_1q61elv,Don’t think market will care what score it gets.,592,2026-01-07 01:28:36
...,...,...,...,...,...,...
12571,nyrktjd,1q8ezhl,t1_nyomllw,"This clone prompt is so entertaining LOL, and ...",1,2026-01-10 12:33:51
12572,nyn3lbw,1q8ezhl,t1_nyn02u2,Well you could set up an MCP server and some t...,3,2026-01-09 20:26:41
12573,nys9wzg,1q8ezhl,t1_nypf774,"I mentioned elsewhere in this thread, but I'm ...",1,2026-01-10 15:47:39
12574,nyp11cl,1q8ezhl,t1_nyp0ht5,yep.,1,2026-01-10 01:55:41


In [85]:
comment_tickers = con.sql("SELECT * FROM comment_tickers").df()
display(comment_tickers)

,comment_id,ticker,direction,confidence,method
0,nyl0vvt,DD,neutral,0.0,caps_context
1,nylf1cy,GOOG,neutral,0.0,caps_plain
2,nylf1cy,AAPL,neutral,0.0,caps_plain
3,nyl98cq,AAPL,neutral,0.5,caps_plain
4,nylbxlb,AAPL,bearish,1.0,caps_context
...,...,...,...,...,...
1029,nynswzm,TSLA,neutral,0.0,caps_context
1030,nyn599y,GPT,bullish,1.0,caps_context
1031,nys9wzg,API,neutral,0.5,caps_plain
1032,nyslrcc,GLP,neutral,0.5,caps_context


In [ ]:
# Check tickers extracted from individual comments
con.sql('''SELECT
            *
        FROM comment_tickers ct
        JOIN comments c ON ct.comment_id = c.comment_id
        WHERE ct.comment_id = 'nyl98cq';
        ''').show()

In [30]:
# Top tickers by mentions
con.sql('''SELECT
          ticker,
          COUNT(*) AS mentions,
          COUNT(DISTINCT comment_id) AS unique_comments
        FROM comment_tickers
        GROUP BY ticker
        HAVING unique_comments >= 10
        ORDER BY mentions DESC
        LIMIT 20;
        ''').show()

┌─────────┬──────────┐
│ ticker  │ mentions │
│ varchar │  int64   │
├─────────┼──────────┤
│ A       │     1161 │
│ IT      │      966 │
│ S       │      758 │
│ FOR     │      666 │
│ YOU     │      644 │
│ T       │      583 │
│ ON      │      566 │
│ ARE     │      485 │
│ BE      │      443 │
│ SO      │      335 │
├─────────┴──────────┤
│ 10 rows  2 columns │
└────────────────────┘



In [31]:
# Bullis vs bearish
con.sql('''SELECT
            ticker,
            SUM(direction = 'bullish') AS bullish,
            SUM(direction = 'bearish') AS bearish
        FROM comment_tickers ct
        JOIN comments c ON ct.comment_id = c.comment_id
        WHERE c.created_utc >= NOW() - INTERVAL '1 day'
        GROUP BY ticker
        ORDER BY bullish DESC;
        ''').show()

┌─────────┬─────────┬─────────┐
│ ticker  │ bullish │ bearish │
│ varchar │ int128  │ int128  │
├─────────┼─────────┼─────────┤
│ A       │     223 │      71 │
│ IT      │     183 │      65 │
│ S       │     133 │      59 │
│ FOR     │     131 │      48 │
│ ON      │     126 │      51 │
│ YOU     │     122 │      33 │
│ T       │     113 │      44 │
│ BE      │      88 │      31 │
│ ARE     │      87 │      40 │
│ AS      │      67 │      17 │
│ ·       │       · │       · │
│ ·       │       · │       · │
│ ·       │       · │       · │
│ IR      │       0 │       0 │
│ FEED    │       0 │       0 │
│ QS      │       0 │       1 │
│ BUD     │       0 │       0 │
│ CENT    │       0 │       1 │
│ SPCE    │       0 │       0 │
│ CDC     │       0 │       0 │
│ ROCK    │       0 │       0 │
│ GME     │       0 │       0 │
│ LC      │       0 │       0 │
├─────────┴─────────┴─────────┤
│ 466 rows          3 columns │
│ (20 shown)                  │
└─────────────────────────────┘



In [32]:
# Weighted by comments
con.sql('''SELECT
  ct.ticker,
  SUM(c.score) AS score_weighted,
  COUNT(*) AS mentions
FROM comment_tickers ct
JOIN comments c ON c.comment_id = ct.comment_id
WHERE c.created_utc >= NOW() - INTERVAL '1 day'
GROUP BY ct.ticker
HAVING COUNT(*) >= 10
ORDER BY score_weighted DESC
LIMIT 20;

        ''').show()

┌─────────┬───────────────────┐
│ ticker  │ weighted_mentions │
│ varchar │      int128       │
├─────────┼───────────────────┤
│ A       │             12223 │
│ ON      │              6866 │
│ IT      │              6316 │
│ FOR     │              5663 │
│ ARE     │              5007 │
│ S       │              4834 │
│ T       │              4446 │
│ YOU     │              4166 │
│ ALL     │              4164 │
│ BE      │              4049 │
├─────────┴───────────────────┤
│ 10 rows           2 columns │
└─────────────────────────────┘



,comment_id,ticker,direction,confidence,method
0,nyl0vvt,DD,neutral,0.0,caps_context
1,nyl4ejb,A,neutral,0.5,caps_repeat
2,nylf1cy,GOOG,neutral,0.0,caps_plain
3,nylf1cy,AAPL,neutral,0.0,caps_plain
4,nyl98cq,AI,neutral,0.5,caps_context
...,...,...,...,...,...
597,nypazr3,VXUS,bullish,1.0,caps_context
598,nypf052,A,neutral,0.5,caps_repeat
599,nyqme1n,AI,bullish,1.0,caps_context
600,nyokbey,AI,neutral,0.5,caps_context


┌────────────┬─────────┬───────────┬────────────┬──────────────┬────────────┬─────────┬────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────┬─────────────────────┐
│ comment_id │ ticker  │ direction │ confidence │    method    │ comment_id │ post_id │ parent_id  │                                                                                                                                                                                                                                    body                                                                           

In [51]:
con.sql('''SELECT
            *
        FROM comment_tickers ct
        JOIN comments c ON ct.comment_id = c.comment_id
        WHERE ct.ticker = 'A'
        LIMIT 500
        ;
        ''').show()

┌────────────┬─────────┬───────────┬────────────┬────────────┬─────────┬────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────┬─────────────────────┐
│ comment_id │ ticker  │ direction │ confidence │ comment_id │ post_id │ parent_id  │                                                                                                                                                  body                                                                                                                                                  │ score │     created_utc     │
│  varchar   │ varchar │  varchar  │   double   │  varchar   │ varchar │  varchar   │                                                                                         

In [12]:
con.sql('''DELETE FROM comment_tickers
        --WHERE ticker IN ('A', 'ON');
        ''')


In [13]:
con.sql('''DELETE FROM post_tickers;
        ''')

In [62]:
con.sql('''ALTER TABLE comment_tickers ADD COLUMN method TEXT; ''')

In [65]:
con.sql('''DESCRIBE comment_tickers;''')


┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ comment_id  │ VARCHAR     │ NO      │ PRI     │ NULL    │ NULL    │
│ ticker      │ VARCHAR     │ NO      │ PRI     │ NULL    │ NULL    │
│ direction   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ confidence  │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ method      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [17]:
con.sql('''ALTER TABLE posts ADD COLUMN IF NOT EXISTS permalink TEXT;
           ALTER TABLE posts ADD COLUMN IF NOT EXISTS url TEXT;
  ''')

In [23]:
con.sql('''DESCRIBE comments;''')

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ comment_id  │ VARCHAR     │ NO      │ PRI     │ NULL    │ NULL    │
│ post_id     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ parent_id   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ body        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ score       │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ created_utc │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [24]:
con.sql('''DESCRIBE posts;''')

┌──────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name  │ column_type │  null   │   key   │ default │  extra  │
│   varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ post_id      │ VARCHAR     │ NO      │ PRI     │ NULL    │ NULL    │
│ subreddit    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ title        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ body         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ score        │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ num_comments │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ created_utc  │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ permalink    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ url          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└──────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [25]:
con.sql('''ALTER TABLE comments ADD COLUMN IF NOT EXISTS subreddit TEXT;''')

In [30]:
con.sql('''DELETE FROM comment_tickers;
DELETE FROM post_tickers;
DELETE FROM comments;
DELETE FROM posts;
''')

In [35]:
con.execute("""
CREATE TABLE IF NOT EXISTS embedding_queue (
  doc_id TEXT PRIMARY KEY,              -- stable unique id
  source_type TEXT NOT NULL,            -- 'post' or 'comment'
  post_id TEXT,                         -- for both posts and comments
  comment_id TEXT,                      -- only for comments
  subreddit TEXT,
  created_utc TIMESTAMP,
  score BIGINT,                         -- upvotes score
  num_comments BIGINT,                  -- only for posts (nullable for comments)
  url TEXT,                             -- reddit URL if available
  title TEXT,                           -- posts only
  text TEXT NOT NULL,                   -- text we embed
  char_len BIGINT NOT NULL,
  inserted_at TIMESTAMP DEFAULT NOW(),
  embedded BOOLEAN DEFAULT FALSE,
  embedded_at TIMESTAMP
);

CREATE INDEX IF NOT EXISTS idx_embedding_queue_embedded ON embedding_queue(embedded);
CREATE INDEX IF NOT EXISTS idx_embedding_queue_created ON embedding_queue(created_utc);


""")

In [36]:
con.close()